In [1]:
import numpyro
import numpyro.distributions as dist
import pandas as pd

import jax
import jax.numpy as jnp
from jax import random
import numpy as np
import numpyro
import numpyro.distributions as dist
from numpyro.infer import NUTS, MCMC
from numpyro.infer.reparam import TransformReparam, LocScaleReparam
import numpyro.distributions.transforms as T

/Users/tsuyos-u/Library/Caches/pypoetry/virtualenvs/bayesian-item-response-theory-uJKz2CJg-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def irt_2pl(resp, N, K, beta=1, **hyper_params):
    hyper_a_sd = hyper_params.get('a', 0.35)
    hyper_b_sd = hyper_params.get('b', 1.0)
        
    theta1 = numpyro.sample("theta1", dist.Normal(0, 1).expand([N]).to_event(1))
    theta = numpyro.deterministic("theta", theta1 - jnp.mean(theta1, axis=0, keepdims=True))

    log_a_sigma = numpyro.sample("log_a_sigma", dist.HalfNormal(hyper_a_sd))
    a = numpyro.sample("a",
                       dist.TruncatedNormal(1., log_a_sigma, low=0).expand([K]).to_event(1)
    )

    b_sd = numpyro.sample("b_sd", dist.HalfNormal(hyper_b_sd))
    with numpyro.handlers.reparam(config={"b": LocScaleReparam()}):
        b = numpyro.sample("b", dist.Normal(0., b_sd).expand([K]).to_event(1))
  
    logit = (a * (theta[:, None] - b))
    p = jax.nn.sigmoid(logit)

    with numpyro.handlers.scale(scale=beta):
        with numpyro.plate("person", N):
            numpyro.sample(
                "obs",
                dist.Bernoulli(probs=p).to_event(1),  # K をイベント
                obs=resp
            )

In [3]:
models = ['gpt-5-mini', 'gemini-2.5-flash', 'claude-sonnet-4']
df = pd.read_excel("data/JSNDI_20250906.xlsx")

for model in models:
    df.loc[df[df[model] == 1].index, 'Model'] = model

In [4]:
subject = 'Code_Comp'
columns = ['Model', 'file', 'Correct']
response = df.query("Cat==@subject")[columns].pivot(index='file', columns='Model')['Correct'].T

In [5]:
rng_key = random.PRNGKey(0),
num_warmup = 2_000
num_samples = 2_000
num_chains = 1

nuts_kernel = NUTS(irt_2pl)
mcmc = MCMC(
    nuts_kernel,
    num_warmup  = num_warmup,
    num_samples = num_samples,
    progress_bar = True,
)

In [6]:
base_key   = random.PRNGKey(0)
chain_keys = random.split(base_key, num_chains)

N, K = response.shape

In [7]:
mcmc.run(chain_keys[0], resp=response.values, N=N, K=K, beta=1)

sample: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4000/4000 [00:01<00:00, 3117.85it/s, 31 steps of size 1.09e-01. acc. prob=0.94]


In [8]:
a = mcmc.get_samples()['a']
b = mcmc.get_samples()['b']
theta = mcmc.get_samples()['theta']

In [9]:
from pathlib import Path
result_path = Path("results")
result_path = result_path / subject
result_path.mkdir(parents=True, exist_ok=True)

file_name = 'a.csv'
pd.DataFrame(a).to_csv(result_path / file_name, index=False)

file_name = 'b.csv'
pd.DataFrame(b).to_csv(result_path / file_name, index=False)

file_name = 'theta.csv'
pd.DataFrame(theta).to_csv(result_path / file_name, index=False)